In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## RAG System Using Llama2 With Hugging Face

In [ ]:
!pip install -r "/content/drive/MyDrive/Colab Notebooks/requirements/requirements.txt"

In [ ]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader,ServiceContext,PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core import PromptTemplate

In [ ]:
documents=SimpleDirectoryReader("/content/drive/MyDrive/Colab Notebooks/gpt_train_tex").load_data()
documents

In [ ]:
SYSTEM_PROMPT = """You are an AI assistant that answers questions in a friendly manner, based on the given source documents. Here are some rules you always follow:
- Generate human readable output, avoid creating output with gibberish text.
- Generate only the requested output, don't include any other language before or after the requested output.
- Never say thank you, that you are happy to help, that you are an AI agent, etc. Just answer directly.
- Generate professional language typically used in business documents in North America.
- Never generate offensive or foul language.
"""

query_wrapper_prompt = PromptTemplate(
    "[INST]<<SYS>>\n" + SYSTEM_PROMPT + "<</SYS>>\n\n{query_str}[/INST] "
)


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.1, "do_sample": False},
    system_prompt=SYSTEM_PROMPT,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16 , "load_in_8bit":True}
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model

In [ ]:
index=VectorStoreIndex.from_documents(documents)
index.storage_context.persist(persist_dir="/content/drive/MyDrive/Colab Notebooks/index")

In [ ]:
index

In [ ]:
query_engine=index.as_query_engine()

In [ ]:
response=query_engine.query("what is energy density in laser powder bed fusion")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.1` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
print(response)

Based on the provided documents, the energy density in laser powder bed fusion (LPMBF) can be calculated using various methods. Here are some of the approaches mentioned in the documents:

1. Energy density = (laser power x scanning speed) / (powder layer thickness x beam overlap) (Document [14])
2. Energy density = (laser power x scanning speed) / (powder layer thickness x beam overlap x number of passes) (Document [15])
3. Energy density = (laser power x scanning speed) / (powder layer thickness x beam overlap x number of passes x absorption coefficient) (Document [16])

From these documents, it is clear that the energy density in LPMBF is a complex quantity that depends on several factors, including the laser power, scanning speed, powder layer thickness, beam overlap, number of passes, and absorption coefficient. The exact value of the energy density can vary depending on the specific application and the properties of the powder being melted.

In summary, the energy density in LPMB

In [ ]:
response=query_engine.query("What is the relation between meltpool dimensions and density?")

In [ ]:
print(response)

Based on the given context, it can be inferred that there is a strong correlation between the melt pool dimensions and density. The melt pool dimensions, such as width and depth, can affect the density of the printed material.

The density of the material can be expressed as a function of the melt pool dimensions using the analytical model provided in the context. The model takes into account the laser power, scan speed, and other process parameters that can affect the melt pool shape and density.

From the experimental results presented in the context, it can be seen that the melt pool dimensions vary depending on the process parameters used. The width and depth of the melt pool are found to be affected by the laser power, scan speed, and other process parameters.

Therefore, it can be concluded that the melt pool dimensions are closely related to the density of the printed material. By controlling the melt pool dimensions, the density of the material can be optimized, leading to impr

In [ ]:
response=query_engine.query("How is phase composition of alloys analysed?")

In [ ]:
print(response)

The phase composition of alloys can be analyzed using various techniques, including:

1. Atomic emission spectroscopy (AES): This technique is used to determine the chemical composition of an alloy by measuring the intensity of atomic emission lines.
2. Inductively coupled plasma mass spectrometry (ICP-MS): This technique is used to determine the chemical composition of an alloy by measuring the mass-to-charge ratio of ions emitted from the sample.
3. X-ray fluorescence (XRF): This technique is used to determine the chemical composition of an alloy by measuring the X-rays emitted by the sample.
4. Scanning electron microscopy (SEM) with energy-dispersive X-ray spectroscopy (EDS): This technique is used to determine the chemical composition of an alloy by analyzing the elemental distribution in the sample using a scanning electron microscope.
5. Transmission electron microscopy (TEM) with energy-dispersive X-ray spectroscopy (EDS): This technique is used to determine the chemical compos

In [ ]:
response=query_engine.query("Explain slicing strategies in laser powder bed fusion" )

In [ ]:
print(response)

Slicing strategies in laser powder bed fusion (LPBF) are techniques used to control the as-solidified bead shape during the additive manufacturing process. The goal of these strategies is to ensure optimal mechanical properties of the final product by controlling the laser parameters, such as speed, power, and beam diameter, during the melting process.

There are several slicing strategies used in LPBF, including:

1. Single-layer or single-track process maps: This method involves measuring the effects of laser parameters on the mechanical properties and track formation by depositing single layers or tracks of powder on a base plate. The crucible method, which was mentioned in the context information, is a type of single-layer process map where the powder is built up in a crucible before being deposited onto the base plate.
2. Multi-layer process maps: This method involves measuring the effects of laser parameters on the mechanical properties and track formation by depositing multiple 

In [ ]:
response=query_engine.query("How do you obtain the temperature distribution in the heat-affected zone?" )

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


In [ ]:
print(response)

To obtain the temperature distribution within the heat-affected zone, we use the equation of heat transfer, which is given by:

$$\rho C_{\text{v}} \frac{\partial T}{\partial t} = K \left( \frac{\partial T}{\partial x} + \frac{\partial T}{\partial y} + \frac{\partial T}{\partial z} \right) + Q$$

where $\rho$ is the density, $C_{\text{v}}$ is the heat capacity, $K$ is the heat conductivity, $T$ is the temperature, and $Q$ is the internal heat generation per unit volume.

We solve this equation along each time step of the single-bead scan with the consideration of nonlinear temperature-dependent Ti-6Al-4V thermo-physical properties. The solidus temperature of the alloy Ti-6Al-4V corresponding to the melt pool boundary is taken as $1873 \text{ K}\left(1600^\circ \text{C}\right)$.

The temperature distribution is obtained by tracking the material state using 3DSIM FLEX simulation software, which solves the thermal diffusion


To obtain the temperature distribution within the heat-affected zone, we use the equation of heat transfer, which is given by:

$$\rho C_{\text{v}} \frac{\partial T}{\partial t} = K \left( \frac{\partial T}{\partial x} + \frac{\partial T}{\partial y} + \frac{\partial T}{\partial z} \right) + Q$$

where $\rho$ is the density, $C_{\text{v}}$ is the heat capacity, $K$ is the heat conductivity, $T$ is the temperature, and $Q$ is the internal heat generation per unit volume.

We solve this equation along each time step of the single-bead scan with the consideration of nonlinear temperature-dependent Ti-6Al-4V thermo-physical properties. The solidus temperature of the alloy Ti-6Al-4V corresponding to the melt pool boundary is taken as $1873 \text{ K}\left(1600^\circ \text{C}\right)$.

The temperature distribution is obtained by tracking the material state using 3DSIM FLEX simulation software, which solves the thermal diffusion

In [ ]:
response=query_engine.query("Why are titanium alloys are popular in additive manufacturing?" )

In [ ]:
print(response)

Titanium alloys are popular in additive manufacturing for several reasons:

1. High strength-to-weight ratio: Titanium alloys have a high strength-to-weight ratio, making them ideal for applications where weight reduction is critical, such as in aerospace and automotive industries.
2. Corrosion resistance: Titanium alloys have excellent corrosion resistance, which is essential in harsh environments, such as in marine or chemical applications.
3. High temperature resistance: Titanium alloys have excellent high-temperature resistance, making them suitable for applications in high-temperature environments, such as in gas turbines or nuclear reactors.
4. Low melting point: Titanium has a low melting point, which makes it easier to melt and manipulate during the additive manufacturing process.
5. Good surface finish: Titanium alloys have a good surface finish, which is important for applications where aesthetics are critical, such as in consumer products.
6. Wide range of alloys: Titanium i

In [ ]:
response=query_engine.query("Why are high strength aluminum alloys difficult to print in laser powder bed fusion?" )

In [ ]:
print(response)

Based on the provided context information, it seems that high strength aluminum alloys are difficult to print in laser powder bed fusion (LPBF) due to the formation of keyhole pores during the process. Keyhole pores are small holes that form in the melt pool during LPBF, and they can lead to a decrease in the mechanical properties of the printed component. The formation of these pores is attributed to the rapid collapse of vapor depressions at the surface of the melt pool, which can trap inert gas, such as argon, in the pores. This can occur during changes in laser scan velocity, such as at laser turn points, and can have a significant impact on the quality of the printed component.
